# Human Evaluation

## What Will We Do? 
* Create an eval prompt template (notebook 4a) 
* Create two different RAG prompt templates and compare results (notebook 4a) 
* Run LLM-As-A-Judge against an eval dataset and grade RAG system response (notebook 4a) 
* **Run Human-Eval to ensure LLM-As-A-Judge is aligned with human preferences (notebook 4b)** 
* **Run Human-Eval and compare GroundTruth with RAG system response (notebook 4b)**
* Run through evaluation with different LLMs (notebook 4c) 

So let's start.

### Install Dependencies

In [360]:
# !conda create -y --name llm-system-eval python=3.11.7
# !conda init && activate llm-system-eval
# !conda install -n llm-system-eval ipykernel --update-deps --force-reinstall -y
# OR
# !pyenv virtualenv 3.11.7 llm-system-eval
# !pyenv activate llm-system-eval

# %pip install -r requirements.txt

### Set Environment Variables

In [3]:
# set variables
from dotenv import load_dotenv, find_dotenv
import os
import sagemaker
import numpy as np
import pandas as pd
import json
import boto3


# loading environment variables that are stored in local file
local_env_filename = 'llm-system-eval.env'
load_dotenv(find_dotenv(local_env_filename),override=True)
os.environ['REGION'] = os.getenv('REGION')
REGION = os.environ['REGION']
os.environ['WORKTEAM_ARN'] = os.getenv('WORKTEAM_ARN')
WORKTEAM_ARN = os.environ['WORKTEAM_ARN']
os.environ['S3_BUCKET_NAME'] = os.getenv('S3_BUCKET_NAME')
S3_BUCKET_NAME = os.environ['S3_BUCKET_NAME']
os.environ['SAGEMAKER_ROLE_ARN'] = os.getenv('SAGEMAKER_ROLE_ARN')
SAGEMAKER_ROLE_ARN = os.environ['SAGEMAKER_ROLE_ARN'] # OR sagemaker.get_execution_role()



# Also initialize the bedrock client so we can call some embedding models!
session = boto3.Session(profile_name='default')
bedrock_runtime_client = boto3.client('bedrock-runtime')

sagemaker_session = sagemaker.Session()
sagemaker_client = boto3.client('sagemaker')
role_name = SAGEMAKER_ROLE_ARN.split("/")[-1]

bucket = sagemaker_session.default_bucket()
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

# Specify ARNs for resources needed to run an text classification job.
ac_arn_map = {
    "us-west-2": "081040173940",
    "us-east-1": "432418664414",
    "us-east-2": "266458841044",
    "eu-west-1": "568282634449"
}
# PreHumanTaskLambdaArn for text classification(single)
prehuman_arn = "arn:aws:lambda:{}:{}:function:PRE-PassThrough".format(
    REGION, ac_arn_map[REGION]
)

# AnnotationConsolidationConfig for text classification(single)
acs_arn = "arn:aws:lambda:{}:{}:function:ACS-PassThrough".format(REGION, ac_arn_map[REGION])

print(prehuman_arn)
print(acs_arn)

arn:aws:lambda:us-east-1:432418664414:function:PRE-PassThrough
arn:aws:lambda:us-east-1:432418664414:function:ACS-PassThrough


# Human Evaluation

## Human Eval1

In [4]:
# Human Eval1: review LLM-as-a-Judge evaluation results with human evaluation results to ensure they align.

# Step 1: Load jsonl file, e.g. 4_answer_validation_opensearch_graded1.jsonl

human_eval_df = pd.read_json('../data/eval-datasets/4_answer_validation_opensearch_graded1.jsonl', lines=True)

eval_criteria = '''Evaluation Criteria (Additive Score, 0-3):
                    1. Context: Award 1 point if the answer uses only information provided in the context, without introducing external or fabricated details.
                    2. Completeness: Add 1 point if the answer addresses all key elements of the question based on the available context, without omissions.
                    3. Conciseness: Add a final point if the answer uses the fewest words possible to address the question and avoids redundancy.
                    '''
# Step 2: Convert to the desired format for GroundTruth
formatted_data = ""
for index, row in human_eval_df.iterrows():
    entry = {
        "source": row["query_text"],
        "response": row["generated_answer"],
        "eval_criteria": eval_criteria,
        "reasoning": row["reasoning"],
        "score": row["score"],
        "retrieved_chunks": str(row["retrieved_chunks"]),
    }
    formatted_data += json.dumps(entry) + "\n"

# Step 3: Save the output as a new JSON file
local_path = '../data/eval-datasets/4_review_judge.json'
with open(local_path, 'w') as f:
    f.write(formatted_data)

print(f"Conversion complete. The file {local_path} is ready.")

# Step 4: Upload the manifest to S3
bucket_name = bucket
prefix = 'genai'
# Upload the file to S3
manifest_s3_uri = sagemaker_session.upload_data(path=local_path, bucket=bucket, key_prefix=prefix)

print(f"File uploaded to {manifest_s3_uri}")

Conversion complete. The file ../data/eval-datasets/4_review_judge.json is ready.
File uploaded to s3://sagemaker-us-east-1-026459568683/genai/4_review_judge.json


In [5]:
# Step 5: Create template for judge eval review
from IPython.display import display, HTML

def make_template(save_fname="4_instructions_judge_review.template"):
    template = """
      <html>
      <head>
        <meta charset="UTF-8" />
        <script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
        <script src="https://assets.crowd.aws/gen-ai-components.js"></script>
        <script>
          document.getElementById('submitButton').onclick = function() {
              document.querySelector('crowd-form').submit();
          };
        </script>
        <style>
          body {
            padding-left: 20px;
            margin-bottom: 20px;
          }
          .highlight {
            background-color: yellow;
          }
          .container {
            margin-bottom: 20px;
          }
          .outer-container {
            display: flex;
            justify-content: space-between;
            width: 100%;
            max-width: 1400px;
          }
          .left-container, .right-container {
            padding: 20px;
            min-width: 75%;
          }
          .right-container {
            border-left: 2px solid #d5dbdb;
          }
        </style>
      </head>
    <body>
    <crowd-form>
      <!-- Prevent crowd-form from creating its own button -->
      <crowd-button form-action="submit" style="display: none;"></crowd-button> 
      <div class="container">
        <h1>Instructions</h1>
        Review the prompt, RAG System response, RAG Context, and evaluation critera. Then complete the tasks on the right panel using your best judgment. 
      </div>
      <br>
      <div class="outer-container">
        <div class="left-container">
          <h2>Evaluation Criteria</h2>
          <div>{{ task.input.eval_criteria }}</div>
          <br>
          <h2>Original Prompt</h2>
          <div>{{ task.input.source }}</div>
          <br>
          <h2>RAG System Response</h2>
          <div>{{ task.input.response }}</div>
          <br>
          <h2>RAG Context</h2>
          <div>{{ task.input.retrieved_chunks }}</div>
          <br> 
          <h2>LLM-as-a-Judge Score</h2>
          <div>{{ task.input.score }}</div>
          <br>
          <h2>LLM-as-a-Judge Reasoning</h2>
          <div>{{ task.input.reasoning }}</div>
          
                
        </div>
        <div class="right-container">
          <h2>Evaluation</h2>
          <div>
            <label for="context">Context:</label>
            <div>
              <input type="radio" id="context1" name="context" value="1" required>
              <label for="context1">1 Point</label><br>
              <input type="radio" id="context2" name="context" value="0">
              <label for="context2">0 Point</label><br>
            </div>
          </div>
          <br>
          <div>
            <label for="completeness">Completeness:</label>
            <div>
              <input type="radio" id="completeness1" name="completeness" value="1" required>
              <label for="completeness1">1 Point</label><br>
              <input type="radio" id="completeness2" name="completeness" value="0">
              <label for="completeness2">0 Point</label><br>
            </div>
          </div>
          <br>
          <div>
            <label for="conciseness">Conciseness:</label>
            <div>
              <input type="radio" id="conciseness1" name="conciseness" value="1" required>
              <label for="conciseness1">1 Point</label><br>
              <input type="radio" id="conciseness2" name="conciseness" value="0">
              <label for="conciseness2">0 Point</label><br>
            </div>
          </div>
          <br>
          <div>
            <label for="issues">Select any issues observed:</label>
            <div>
              <input type="checkbox" id="grammar" name="issues" value="grammar">
              <label for="grammar">Grammar issues</label><br>
              <input type="checkbox" id="factual" name="issues" value="factual">
              <label for="factual">Factual inaccuracies</label><br>
              <input type="checkbox" id="missing" name="issues" value="missing">
              <label for="missing">Missing information</label><br>
              <input type="checkbox" id="other" name="issues" value="other">
              <label for="other">Other</label>
            </div>
          </div>
          <br>
          <div>
           <crowd-button id="submitButton">Submit</crowd-button>
          </div
        </div>
      </div>
    </crowd-form>
    </body>
    </html>
    """
    with open(save_fname, "w") as f:
        f.write(template)

# Create the template file locally
local_path ='../data/eval-datasets/4_instructions_judge_review.template'
make_template(save_fname=local_path)

# Upload the file to S3
template_s3_uri = sagemaker_session.upload_data(path=local_path, bucket=bucket, key_prefix=prefix)

print(f"Template file uploaded to {template_s3_uri}")

Template file uploaded to s3://sagemaker-us-east-1-026459568683/genai/4_instructions_judge_review.template


In [6]:
# Step 6: Create Human Eval job with GroundTruth: review LLM as a Judge evaluation results with human evaluation results
import datetime

now = datetime.datetime.now()
timestamp_str = now.strftime("%Y%m%d-%H%M%S")
labeling_job_name = "rag-review-llm-judge-" + timestamp_str

sagemaker_client.create_labeling_job(
    LabelingJobName=labeling_job_name,
    LabelAttributeName='label',
    InputConfig={
        'DataSource': {
            'S3DataSource': {
                'ManifestS3Uri': manifest_s3_uri #Enter S3 URI of Input Data Json
            }
        }
    },
    OutputConfig={
        'S3OutputPath': f's3://{bucket}/{prefix}/output/' #Enter S3 URI of Output folder
    },
    RoleArn=SAGEMAKER_ROLE_ARN, #Enter IAM Sagemaker Execution Role here,
    HumanTaskConfig={
        'WorkteamArn': WORKTEAM_ARN, #Enter Workteam ARN
        'UiConfig': {
            'UiTemplateS3Uri': template_s3_uri #Enter S3 URI of UI template
        },
        'PreHumanTaskLambdaArn': prehuman_arn,
        'TaskKeywords': [
            'QnA',
        ],
        'TaskTitle': labeling_job_name,
        'TaskDescription': "Review the LLM-as-a-Judge results using the dimensions on the side",
        'NumberOfHumanWorkersPerDataObject': 1,
        'TaskTimeLimitInSeconds': 60*30,
        'TaskAvailabilityLifetimeInSeconds': 60*60*24*10,
        'MaxConcurrentTaskCount': 100,
        'AnnotationConsolidationConfig': {
            'AnnotationConsolidationLambdaArn': acs_arn
        }
    })

{'LabelingJobArn': 'arn:aws:sagemaker:us-east-1:026459568683:labeling-job/rag-review-llm-judge-20240827-233822',
 'ResponseMetadata': {'RequestId': '700b4ee3-ad60-4d9c-aee4-786aaa0cf8f1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '700b4ee3-ad60-4d9c-aee4-786aaa0cf8f1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '111',
   'date': 'Wed, 28 Aug 2024 03:38:22 GMT'},
  'RetryAttempts': 0}}

In [7]:
# Step 7: Do Human evaluation
workforce = sagemaker_client.describe_workforce(WorkforceName="default")
worker_portal_url = 'https://' + workforce["Workforce"]["SubDomain"]


# Display the URL and instructions
display(HTML(f"""
<body>
<h4>Review LLM-as-a-Judge results</h4>
<p>Please complete the human evaluation tasks available in the labeling portal.</p>
<p><a href="{worker_portal_url}">{worker_portal_url}</a>
<p><b>Ensure all tasks are completed before proceeding to the next steps in this notebook.<b></p>
<body>
"""))

In [14]:
# Step 8: monitor the job's progress
sagemaker_client.describe_labeling_job(LabelingJobName=labeling_job_name)
# stop labeling job
# sagemaker_client.stop_labeling_job(LabelingJobName=labeling_job_name)

{'LabelingJobStatus': 'Completed',
 'LabelCounters': {'TotalLabeled': 24,
  'HumanLabeled': 24,
  'MachineLabeled': 0,
  'FailedNonRetryableError': 0,
  'Unlabeled': 0},
 'CreationTime': datetime.datetime(2024, 8, 27, 23, 38, 23, 55000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 8, 28, 11, 2, 40, 498000, tzinfo=tzlocal()),
 'JobReferenceCode': '026459568683QNSoxT1GelBEmC4k',
 'LabelingJobName': 'rag-review-llm-judge-20240827-233822',
 'LabelingJobArn': 'arn:aws:sagemaker:us-east-1:026459568683:labeling-job/rag-review-llm-judge-20240827-233822',
 'LabelAttributeName': 'label',
 'InputConfig': {'DataSource': {'S3DataSource': {'ManifestS3Uri': 's3://sagemaker-us-east-1-026459568683/genai/4_review_judge.json'}}},
 'OutputConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-026459568683/genai/output/',
  'KmsKeyId': ''},
 'RoleArn': 'arn:aws:iam::026459568683:role/demo-SagemakerNotebookIAMRole-026459568683',
 'StoppingConditions': {'MaxPercentageOfInputDatasetLabeled': 1

In [17]:
# # Step 9: Compare HumanEval with LLM-as-a-Judge results
def load_json_from_s3(bucket, key):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    data = json.loads(obj['Body'].read().decode('utf-8'))
    return data

source_bucket_name = bucket
key = f'{prefix}/output/{labeling_job_name}/manifests/output/output.manifest'

# Download the manifest file
response = s3_client.get_object(Bucket=source_bucket_name, Key=key)
content = response['Body'].read().decode('utf-8')

# Split the content into lines
lines = content.strip().split('\n')

# Parse each line as JSON and create a list of dictionaries
data = [json.loads(line) for line in lines]
manifest_df = pd.DataFrame(data)

for index, row in manifest_df.iterrows():
    label_metadata = row['label-metadata']
    key = label_metadata['worker-response-ref'].replace(f's3://{bucket}/','')
    data = load_json_from_s3(bucket, key)
    completeness = 0
    conciseness = 0
    context = 0
    for answer in data["answers"]:
        human_eval_score = 0
        if answer['answerContent']['completeness']['1'] == True:
            human_eval_score += 1
            completeness = 1

        if answer['answerContent']['conciseness']['1'] == True:
            human_eval_score += 1
            conciseness = 1
        
        if answer['answerContent']['context']['1'] == True:
            human_eval_score += 1
            context = 1
        
    manifest_df.at[index, 'completeness'] = completeness
    manifest_df.at[index, 'conciseness'] = conciseness
    manifest_df.at[index, 'context'] = context
    manifest_df.at[index, 'human_eval_score'] = human_eval_score

manifest_df = manifest_df.drop(columns=['label','eval_criteria','retrieved_chunks','label-metadata'])

# Convert the dataframe to an HTML table with escaped HTML content
table_html = manifest_df.to_html(index=False, classes='table table-striped', escape=False)
display(HTML(table_html))

source,response,reasoning,score,completeness,conciseness,context,human_eval_score
I'm using version 2.1 of open search and trying to use zstd compression. Why isn't it working?,There is not enough information available to answer this question. The context does not mention why zstd compression is not working for your specific case with OpenSearch version 2.1. The context only provides information about the zstd compression codec being introduced in OpenSearch 2.9 and later versions.,Context:\n1. The context does not provide any information about why zstd compression is not working for OpenSearch version 2.1. It only mentions that zstd compression was introduced in OpenSearch 2.9 and later versions.\n\nCompleteness:\n1. The answer correctly states that there is not enough information in the context to explain why zstd compression is not working for OpenSearch 2.1.\n2. The answer addresses the key elements of the question by acknowledging that the context does not cover version 2.1 and only provides information about compression codecs available in 2.9 and later versions.\n\nConciseness:\n1. The answer is concise and does not contain any redundant information. It directly states that the context lacks the necessary information to answer the question.,3,0.0,1.0,1.0,2.0
"I'm trying to set up this new aggregate view thing for saved objects in OpenSearch Dashboards, but I'm worried about messing up our existing multi-tenancy setup. The docs mention something about tenant indexes and a kibana_server role. How do I make sure I don't break anything when I turn this feature on? And what's the deal with not being able to turn it off once it's enabled","The docs mention that the aggregate view feature for saved objects in OpenSearch Dashboards is an experimental feature that must be enabled using a feature flag in the opensearch_dashboards.yml file. Once enabled, it cannot be disabled without potentially causing issues like data loss or breaking tenant-to-tenant functionality. The docs also state that this feature should only be used in a new test cluster, not in an existing production cluster. As for the kibana_server role, it is required for OpenSearch Dashboards to perform necessary operations and has permissions defined for various indexes and actions. There is not enough information provided about how to ensure the existing multi-tenancy setup is not broken when enabling the aggregate view feature.","Context:\n1. The context mentions that the aggregate view feature for saved objects in OpenSearch Dashboards is an experimental feature that must be enabled using a feature flag. It also states that once enabled, it cannot be disabled without potentially causing issues like data loss or breaking tenant-to-tenant functionality. The context does not provide any information on how to ensure the existing multi-tenancy setup is not broken when enabling this feature.\n\nCompleteness:\n1. The answer addresses the key elements of the question by mentioning that the aggregate view feature is experimental, must be enabled using a feature flag, and cannot be disabled once enabled without potential issues. However, it does not provide any information on how to ensure the existing multi-tenancy setup is not broken, as mentioned in the question.\n\nConciseness:\n1. The answer is concise and does not contain any redundant information.",2,1.0,1.0,1.0,3.0
What software license does Opensearch use?,There is not enough information available to answer this question,"Context:\n1. The context does not provide any information about the software license used by OpenSearch. Therefore, I cannot award a point for this criterion.\n\nCompleteness:\n2. The generated answer correctly states that there is not enough information available to answer the question based on the provided context. Therefore, I will award 1 point for completeness.\n\nConciseness:\n3. The generated answer is concise and uses the fewest words possible to address the question without redundancy. I

## Human Eval2

In [32]:
# Human Eval2: compare ground truth against generated anwsers to ensure it is in line with human preferences

# Step 1: Load jsonl file, e.g. 4_answer_validation_opensearch_graded1.jsonl

human_eval_df = pd.read_json('../data/eval-datasets/4_answer_validation_opensearch_graded1.jsonl', lines=True)

# Step 2: Convert to the desired format
import re
ANSWER_PATTERN = r'<question_answer>(.*?)</question_answer>'

# Strip out the portion of the response with regex.
def extract_with_regex(response, regex):
    matches = re.search(regex, response, re.DOTALL)
    # Extract the matched content, if any
    return matches.group(1).strip() if matches else None

formatted_data = ""
for index, row in human_eval_df.iterrows():
    entry = {
        "source": row["query_text"],
        "responses": [extract_with_regex(row["groundtruth_answer"], ANSWER_PATTERN), row["generated_answer"]]
        
    }
    formatted_data += json.dumps(entry) + "\n"

# Step 3: Save the output as a new JSON file
with open('../data/eval-datasets/4_gt_input_manifest_textranking.json', 'w') as f:
    f.write(formatted_data)

print("Conversion complete. The file '4_gt_input_manifest_textranking.json' is ready.")

# Upload the manifest to S3
bucket = sagemaker_session.default_bucket()
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

bucket_name = bucket
prefix = 'genai'
local_path = '../data/eval-datasets/4_gt_input_manifest_textranking.json'

# Upload the file to S3
manifest_s3_uri = sagemaker_session.upload_data(path=local_path, bucket=bucket, key_prefix=prefix)

print(f"File uploaded to {manifest_s3_uri}")

Conversion complete. The file '4_gt_input_manifest_textranking.json' is ready.
File uploaded to s3://sagemaker-us-east-1-026459568683/genai/4_gt_input_manifest_textranking.json


In [33]:
# Create Review Template
from IPython.display import display, HTML

def make_template(save_fname="instructions_tr.template"):
    template = """
    <html>
      <head>
        <meta charset="UTF-8" />
        <link rel="stylesheet" href="https://assets.crowd.aws/css/gen-ai-components.css" />
        <link rel="icon" href="data:image/svg+xml,<svg xmlns=%22http://www.w3.org/2000/svg%22 viewBox=%220 0 100 100%22><text y=%22.9em%22 font-size=%2290%22>&#129351;</text></svg>" />
        <title>Ranking Tool</title>
        <script src="https://assets.crowd.aws/gen-ai-components.js"></script>
      </head>

      <body>
        <div>
          <crowd-text-ranking
            crowd-form-element-id="crowd-form-submit"
            instructions='Rank the following responses from a RAG system according to the evaluation criteria.'
            ordinal-ranking-dimensions='[{"name":"Accuracy","allowTie":true},{"name":"Clarity","allowTie":true}]'
            text='{{ task.input.source }}'
            responses='{{ task.input.responses | to_json }}' />
        </div>
        <crowd-form id="crowd-form-submit" style="display: none"></crowd-form>
        <script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
      </body>
    </html>
    """
    with open(save_fname, "w") as f:
        f.write(template)

# Create the template file locally
local_path ='../data/eval-datasets/4_instructions_tr.template'
make_template(save_fname=local_path)

# Upload the file to S3
template_s3_uri = sagemaker_session.upload_data(path=local_path, bucket=bucket, key_prefix=prefix)

print(f"Template file uploaded to {template_s3_uri}")

Template file uploaded to s3://sagemaker-us-east-1-026459568683/genai/4_instructions_tr.template


In [34]:
import datetime

now = datetime.datetime.now()
timestamp_str = now.strftime("%Y%m%d-%H%M%S")
labeling_job_name = "rag-system-eval-" + timestamp_str

sagemaker_client.create_labeling_job(
    LabelingJobName=labeling_job_name,
    LabelAttributeName='label',
    InputConfig={
        'DataSource': {
            'S3DataSource': {
                'ManifestS3Uri': manifest_s3_uri #Enter S3 URI of Input Data Json
            }
        }
    },
    OutputConfig={
        'S3OutputPath': f's3://{bucket}/{prefix}/output/' #Enter S3 URI of Output folder
    },
    RoleArn=SAGEMAKER_ROLE_ARN, #Enter IAM Sagemaker Execution Role here,
    HumanTaskConfig={
        'WorkteamArn': WORKTEAM_ARN, #Enter Workteam ARN
        'UiConfig': {
            'UiTemplateS3Uri': template_s3_uri #Enter S3 URI of UI template
        },
        'PreHumanTaskLambdaArn': prehuman_arn,
        'TaskKeywords': [
            'QnA',
        ],
        'TaskTitle': labeling_job_name,
        'TaskDescription': "Rank the responses provided by the Models based on the Dimensions",
        'NumberOfHumanWorkersPerDataObject': 1,
        'TaskTimeLimitInSeconds': 60*30,
        'TaskAvailabilityLifetimeInSeconds': 60*60*24*10,
        'MaxConcurrentTaskCount': 100,
        'AnnotationConsolidationConfig': {
            'AnnotationConsolidationLambdaArn': acs_arn
        }
    })

{'LabelingJobArn': 'arn:aws:sagemaker:us-east-1:026459568683:labeling-job/rag-system-eval-20240828-140613',
 'ResponseMetadata': {'RequestId': 'b029fd7b-60b4-456f-90f7-82a1c608187e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b029fd7b-60b4-456f-90f7-82a1c608187e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '106',
   'date': 'Wed, 28 Aug 2024 18:06:13 GMT'},
  'RetryAttempts': 0}}

In [35]:
workforce = sagemaker_client.describe_workforce(WorkforceName="default")
worker_portal_url = 'https://' + workforce["Workforce"]["SubDomain"]


# Display the URL and instructions
display(HTML(f"""
<body>
<h4>Gather human preference data</h4>
<p>Please complete the human evaluation tasks available in the labeling portal.</p>
<p><a href="{worker_portal_url}">{worker_portal_url}</a>
<p><b>Ensure all tasks are completed before proceeding to the next steps in this notebook.<b></p>
<body>
"""))

In [40]:
# monitor the job's progress
sagemaker_client.describe_labeling_job(LabelingJobName=labeling_job_name)
# sagemaker_client.stop_labeling_job(LabelingJobName=labeling_job_name)

{'LabelingJobStatus': 'Completed',
 'LabelCounters': {'TotalLabeled': 24,
  'HumanLabeled': 24,
  'MachineLabeled': 0,
  'FailedNonRetryableError': 0,
  'Unlabeled': 0},
 'CreationTime': datetime.datetime(2024, 8, 28, 14, 6, 14, 76000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 8, 28, 14, 38, 53, 84000, tzinfo=tzlocal()),
 'JobReferenceCode': '026459568683UKqXoQllz4To3x2V',
 'LabelingJobName': 'rag-system-eval-20240828-140613',
 'LabelingJobArn': 'arn:aws:sagemaker:us-east-1:026459568683:labeling-job/rag-system-eval-20240828-140613',
 'LabelAttributeName': 'label',
 'InputConfig': {'DataSource': {'S3DataSource': {'ManifestS3Uri': 's3://sagemaker-us-east-1-026459568683/genai/4_gt_input_manifest_textranking.json'}}},
 'OutputConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-026459568683/genai/output/',
  'KmsKeyId': ''},
 'RoleArn': 'arn:aws:iam::026459568683:role/demo-SagemakerNotebookIAMRole-026459568683',
 'StoppingConditions': {'MaxPercentageOfInputDatasetLabel

In [46]:
import boto3
import json
import pandas as pd
from IPython.display import display, HTML

s3_client = boto3.client('s3')

def load_json_from_s3(bucket, key):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    data = json.loads(obj['Body'].read().decode('utf-8'))
    return data

source_bucket_name = bucket
key = f'{prefix}/output/{labeling_job_name}/manifests/output/output.manifest'

# Download the manifest file
response = s3_client.get_object(Bucket=source_bucket_name, Key=key)
content = response['Body'].read().decode('utf-8')

# Split the content into lines
lines = content.strip().split('\n')

# Parse each line as JSON and create a list of dictionaries
data = [json.loads(line) for line in lines]
manifest_df = pd.DataFrame(data)

def highlight_response(original, ranked, rank, is_changed, has_same_rank):
    if is_changed:
        return f'<span style="background-color: yellow;">{ranked}</span>'
    elif has_same_rank:
        return f'<span style="background-color: #CCFFCC;">{ranked}</span>'
    return ranked

for index, row in manifest_df.iterrows():
    responses = row['responses']
    # print(f'responses: {responses}')
    label_metadata = row['label-metadata']
    key = label_metadata['worker-response-ref'].replace(f's3://{bucket}/','')
    data = load_json_from_s3(bucket, key)
    for answer in data["answers"]:
        for dimension in answer["answerContent"]["ordinalRankingDimensions"]:
            rankings = dimension["responseRankings"]
            
            # Store the original rankings in a new field
            dimension["responseIndex"] = rankings
            # print(f'rankings: {rankings}')
            
            # Create a list to store the response texts based on rankings
            ranked_responses = [responses[i - 1] for i, _ in sorted(enumerate(rankings, start=1), key=lambda x: x[1])]
            
            # Identify responses with the same ranking
            same_rank_indices = [i for i, rank in enumerate(rankings) if rankings.count(rank) > 1]
            
            # Compare ranked_responses with original responses and highlight differences
            highlighted_responses = []
            for i, (orig, ranked) in enumerate(zip(responses, ranked_responses)):
                is_changed = orig != ranked
                has_same_rank = i in same_rank_indices
                highlighted_responses.append(highlight_response(orig, ranked, rankings[i], is_changed, has_same_rank))
            
            dimension["responseRankings"] = highlighted_responses
            if dimension['name'] == 'Accuracy':
                manifest_df.at[index, 'Accuracy'] = ''.join(highlighted_responses)
            if dimension['name'] == 'Clarity':
                manifest_df.at[index, 'Clarity'] = ''.join(highlighted_responses)

manifest_df = manifest_df.drop(columns=['label','label-metadata'])

# Convert the dataframe to an HTML table with escaped HTML content
table_html = manifest_df.to_html(index=False, classes='table table-striped', escape=False)
display(HTML(table_html))

source,responses,Accuracy,Clarity
I'm using version 2.1 of open search and trying to use zstd compression. Why isn't it working?,"[There is not enough information available to answer this question. The context does not mention why zstd compression is not working for OpenSearch version 2.1. The context provides information about compression codecs available in OpenSearch 2.9 and later versions, but does not cover version 2.1., There is not enough information available to answer this question. The context does not mention why zstd compression is not working for your specific case with OpenSearch version 2.1. The context only provides information about the zstd compression codec being introduced in OpenSearch 2.9 and later versions.]","There is not enough information available to answer this question. The context does not mention why zstd compression is not working for OpenSearch version 2.1. The context provides information about compression codecs available in OpenSearch 2.9 and later versions, but does not cover version 2.1.There is not enough information available to answer this question. The context does not mention why zstd compression is not working for your specific case with OpenSearch version 2.1. The context only provides information about the zstd compression codec being introduced in OpenSearch 2.9 and later versions.","There is not enough information available to answer this question. The context does not mention why zstd compression is not working for OpenSearch version 2.1. The context provides information about compression codecs available in OpenSearch 2.9 and later versions, but does not cover version 2.1.There is not enough information available to answer this question. The context does not mention why zstd compression is not working for your specific case with OpenSearch version 2.1. The context only provides information about the zstd compression codec being introduced in OpenSearch 2.9 and later versions."
"I'm trying to set up this new aggregate view thing for saved objects in OpenSearch Dashboards, but I'm worried about messing up our existing multi-tenancy setup. The docs mention something about tenant indexes and a kibana_server role. How do I make sure I don't break anything when I turn this feature on? And what's the deal with not being able to turn it off once it's enabled","[The docs mention that the aggregate view feature for saved objects in OpenSearch Dashboards is an experimental feature that must be enabled using a feature flag in the opensearch_dashboards.yml file. Once enabled, it cannot be disabled without potentially causing issues like data loss or breaking tenant-to-tenant functionality. The docs also state that this feature should only be used in a new test cluster, not in an existing production cluster. There is no information provided about how to ensure the existing multi-tenancy setup is not broken when enabling this feature., The docs mention that the aggregate view feature for saved objects in OpenSearch Dashboards is an experimental feature that must be enabled using a feature flag in the opensearch_dashboards.yml file. Once enabled, it cannot be disabled without potentially causing issues like data loss or breaking tenant-to-tenant functionality. The docs also state that this feature should only be used in a new test cluster, not in an existing production cluster. As for the kibana_server role, it is required for OpenSearch Dashboards to perform necessary operations and has permissions defined for various indexes and actions. There is not enough information provided about how to ensure the existing multi-tenancy setup is not broken when enabling the aggregate view feature.]","The docs mention that the aggregate view feature for saved objects in OpenSearch Dashboards is an experimental feature that must be enabled using a feature flag in the opensearch_dashboards.yml file. Once enabled, it cannot be disabled without potentially causing issues like data loss or breaking tenant-to-t

# Takeaways
* With the first human evaluation, we ensure alignment between human preferences and the LLM-as-a-Judge, which is more scalable.
* With the second human evaluation, we assess if the generated response and ground truth is aligned with the human preference.
* An important consideration is how you want to improve the system. Being able to quickly find poor performing prompts / inputs is key to improving it. In a feedback loop, it’s common to take poor performing examples, correct them, and use them as dynamic few shot examples in future prompts. If you’re going down the model training path, it’s important to collect data points in pairs. RLHF whether that’s PPO or DPO relies on pair wise data when going through data cycles. If you output diverse pairs of answers, it saves data annotators time because they are rating their preference between pairs vs. having to hand write the answer they prefer.
